# Transformation steps for data pipeline


In [7]:
#Import packages

#! pip3 install unidecode
#! pip3 install SQLAlchemy

import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt


In [8]:
#Connecting to the database

user_Name = 'root'
user_Pass = 'xxxxxxx'

# Instantiate connection
db_conn = mysql.connect(host='localhost', port=int(3306), user=user_Name, passwd=user_Pass, db='507_final_proj', local_infile=True)

In [9]:
#Display tables in database
tbl_names = pd.read_sql('SHOW TABLES', db_conn)

display(tbl_names)
print(type(tbl_names))

/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/1033794614.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tbl_names = pd.read_sql('SHOW TABLES', db_conn)


,Tables_in_507_final_proj
0,country_map
1,emissions_gross
2,emissions_tempy
3,etp_view
4,iso
5,iso_tempy
6,population
7,population_tempy
8,population_trans
9,temperature


<class 'pandas.core.frame.DataFrame'>


In [10]:
#Extract data from each table - emissions

emissions_query = "SELECT * FROM emissions_gross;"
emissions_df = pd.read_sql(emissions_query, db_conn)

emissions_df

/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/3015122745.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emissions_df = pd.read_sql(emissions_query, db_conn)


,eg_id,country,iso3,record_year,total,coal,oil,gas,cement,flaring,other,per_capita,country_code


In [11]:
#Extract data from each table - temperature
temperature_query = "SELECT * FROM temperature;"
temperature_df = pd.read_sql(temperature_query, db_conn)

temperature_df

/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/1901485503.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  temperature_df = pd.read_sql(temperature_query, db_conn)


,domain_code,domain,area_code,country,element_code,element,month_code,month_name,year_code,record_year,unit,temp,flag,flag_desc,country_code


In [12]:
#Extract data from each table - population
worldpop_query = "SELECT * FROM population_trans;"
worldpop_df = pd.read_sql(worldpop_query, db_conn)

worldpop_df

/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/2785358314.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  worldpop_df = pd.read_sql(worldpop_query, db_conn)


,pop_trans_id,pop_rank,iso3,country,capital,continent,area,density,grow_rate,pop_perc,year,population,country_code


In [13]:
#Transformation step - this is for country, year, total, temperature, and population only

Transform1 = pd.read_sql("""
SELECT e.Country, e.Record_year, e.Total, 
t.temp AS Temperature, 
p.population AS Population
FROM emissions_gross e
JOIN temperature t ON e.country = t.country AND e.record_year = t.record_year
JOIN population_trans p ON e.country = p.country
ORDER BY e.country, e.record_year""", db_conn)

Transform1

/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/506907229.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Transform1 = pd.read_sql("""


,Country,Record_year,Total,Temperature,Population


In [14]:
#General transformation to include as much raw data to show relationships between all datasets for emissions, temperature, and population.

Transform2 = pd.read_sql("""
SELECT e.total AS emissions_total, e.country AS emissions_country, e.record_year AS emissions_year,
t.country AS temperature_country, t.temp AS temperature_temp, t.record_year AS temperature_year,
p.year AS population_year, p.pop_perc AS population_percentage
FROM emissions_gross e
INNER JOIN temperature t ON e.country_code = t.country_code AND e.record_year = t.record_year
INNER JOIN population_trans p ON e.country_code = p.country_code AND e.record_year = p.year
ORDER BY emissions_country, emissions_year;
""", db_conn)

Transform2

/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/857108250.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Transform2 = pd.read_sql("""


,emissions_total,emissions_country,emissions_year,temperature_country,temperature_temp,temperature_year,population_year,population_percentage


In [15]:
#Main Transformation - depicts emission factors, temperature, and population factors

Transform3 = pd.read_sql("""
SELECT e.country, e.record_year, e.total, e.coal, e.oil, e.gas, e.cement, e.flaring, e.other, e.per_capita, 
t.temp AS Temperature, 
p.density, p.grow_rate, p.pop_perc
FROM emissions_gross e
JOIN temperature t ON e.country = t.country AND e.record_year = t.record_year
JOIN population_trans p ON e.country = p.country
""", db_conn)

Transform3

/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/3676722192.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Transform3 = pd.read_sql("""


,country,record_year,total,coal,oil,gas,cement,flaring,other,per_capita,Temperature,density,grow_rate,pop_perc


In [16]:
#Create a view for data security purposes and hide complexity of queries

# Create a cursor object
cursor = db_conn.cursor()

# Execute the CREATE VIEW query
create_view_query = """
CREATE VIEW etp_view AS
SELECT e.country, e.record_year, e.total, e.coal, e.oil, e.gas, e.cement, e.flaring, e.other, e.per_capita, 
t.temp AS Temperature, 
p.density, p.grow_rate, p.pop_perc
FROM emissions_gross e
JOIN temperature t ON e.country = t.country AND e.record_year = t.record_year
JOIN population_trans p ON e.country = p.country
"""
cursor.execute(create_view_query)

# Commit the changes to the database
db_conn.commit()

#References: OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/


OperationalError: (1050, "Table 'etp_view' already exists")

In [17]:
#Query with the View - to result highest total emissions and temperature recorded for each country every year

View_query = pd.read_sql("""
SELECT country, 
record_year, MAX(total) AS max_emission, 
MAX(Temperature) AS max_temperature
FROM etp_view
GROUP BY country, record_year
""", db_conn)

View_query

#reference: Beaulieu, A. (2020). Learning SQL: Generate, manipulate, and retrieve data (3rd ed.). O'Reilly.


/var/folders/vb/3r12k85n0319xy1yn6hprksw0000gn/T/ipykernel_95814/1421056473.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  View_query = pd.read_sql("""


,country,record_year,max_emission,max_temperature
